In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# 모든 열 보기 설정
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.0f}'.format
# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [3]:
df = pd.read_csv('./final_부동산.csv',encoding='utf-8')
df

,기준년월,가명식별자,외감구분,설립일자,종업원수,주소지시군구,상장일자,상장폐지일자,유동자산,비유동자산,당좌자산,재고자산,유형자산,재공품,현금,현금등가물,상품유가증권,현금성자산,매출채권,매출채권(전기),매출채권처분손실(당기),무형자산,투자자산,자산총계,자산총계(전기),유동부채,단기차입금,차입금,매입채무,비유동부채,부채총계,자기자본(납입자본금),자본잉여금,납입자본,이익잉여금,자본조정,기타포괄손익누계액,유보금,자본총계,전기자본총계,매출액,전기매출액,매출원가,매출총이익,판매비와관리비,법인세비용차감전 순이익,전기법인세차감전순이익,법인세,계속사업이익,중단산업손익,금융비용,영업손익,전기영업이익,영업외수익,영업외비용,법인세차감전순이익,당기순이익,당기순이익(전기),현금흐름,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,부채상환계수,영업이익이자보상배율,이자비용,사채이자(당기),이자보상배율,적립금비율,EBIT,EBITDA,청산가치율,청산가치,순운전자본,순차입금,재무비율_총자산증가율,재무비율_부채비율,재무비율_자기자본비율,재무비율_유동비율,재무비율_차입금의존도,재무비율_매출액증가율,재무비율_영업이익율,재무비율_당기순이익율,재무비율_매출원가율,재무비율_판관비율,재무비율_자기자본이익률(ROE),재무비율_매출채권회전율,재무비율_재고자산회전율,재무비율_매입채무회전율,재무비율_총자산회전율,재무비율_총자산순이익률,재무비율_유동자산증가율,재무비율_유형자산증가율,단기차입금의존도,당좌비율,순차입금비율,순운전자본회전율,총자본회전율,자기자본순이익율,매출총이익율,EBITDA마진율,영업이익증가율,당기순이익증가율,EBITDA증가율,OCF/매출액비용,부채상환계수.1,차입금/EBITDA,EBITDA/금융비용,사업장소유여부,소유건축물건수,소유건축물실거래가합계,사업장권리침해여부,소유건축물권리침해여부,기업신용공여연체과목수(일보)(미해제),기업신용공여연체과목수(일보)(3개월내유지)(해제포함),기업신용공여연체과목수(일보)(6개월내유지)(해제포함),기업신용공여연체과목수(일보)(1년내유지)(해제포함),기업신용공여연체과목수(일보)(3년내유지)(해제포함),기업신용공여연체과목수(일보)(3개월내발생)(해제포함),기업신용공여연체과목수(일보)(6개월내발생)(해제포함),기업신용공여연체과목수(일보)(1년내발생)(해제포함),기업신용공여연체과목수(일보)(3년내발생)(해제포함),기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함),기업신용공여30일이상연체과목수(일보)(해제포함),기업신용공여30일이상연체과목수(일보)(미해제),기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함),기업신용공여30일이상연체과목수(일보)(이자연체)(미해제),기업신용공여연체기관수(일보)(미해제),기업신용공여30일이상연체기관수(일보)(해제포함),기업신용공여30일이상연체기관수(일보)(미해제),기업신용공여30일이상연체기관수(일보)(연체)(해제포함),기업신용공여30일이상연체기관수(일보)(이자연체)(미해제),기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함),기업신용공여연체최장연체일수(일보)(3개월내유지)(해제포함),기업신용공여연체최장연체일수(일보)(6개월내유지)(해제포함),기업신용공여연체최장연체일수(일보)(1년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(3년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(5년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(3개월내발생)(해제포함),기업신용공여연체최장연체일수(일보)(6개월내발생)(해제포함),기업신용공여연체최장연체일수(일보)(1년내발생)(해제포함),기업신용공여연체최장연체일수(일보)(3년내발생)(해제포함),기업신용공여연체최장연체일수(일보)(5년내발생)(해제포함),신용도판단공공정보건수(CIS)(5년내발생)(해제포함),신용도판단정보공공정보건수(CIS)(미해제),신용도판단정보공공정보건수(관련인제외)(CIS)(당월유지)(해제포함),"공공정보(국세,지방세,관세체납)건수(CIS)(미해제)","공공정보(국세,지방세,관세체납)건수(CIS)(5년내발생)","공공정보(국세,지방세,관세체납,고용산재체납)건수(CIS)(미해제)","공공정보(국세,지방세,관세체납,고용산재체납)건수(CIS)(5년내발생)","신용도판단정보공공정보최근발생일자로부터경과일수(CIS)(해제,삭제)","신용도판단정보공공정보최근해제일자로부터경과일수(CIS)(해제,삭제)",기업신용평가등급(구간화),모형개발용Performance(향후1년내부도여부),업종(대분류)
0,20210801,41294,1,20020830,63,"11,680",NaN,NaN,4406362501,2643245525,1684771733,2721590768,2126501078,67852456,0,0,0,24565275,515949640,909118333,0,0,478016874,7049608026,6735311108,1778620149,676428759,998606328,0,61218398,1839838547,256895000,0,256895000,2421662507,-12293178,-9379458,2421662507,2656884871,2425963213,2717487867,1790577446,1578601892,1138885975,154266460,984619515,253882089,59513932,926612530,0,31406384,984619515,253882089,96963470,95456523,986126462,926612530,664204007,-477677468,360587698,-1178644105,340378939,5,1,32788480,0,1,934,861217368,970761099,93,6532863579,2627742352,730196664,4,69,38,248,32,52,36,34,58,6,0,4,1,0,0,13,0,59,10,95,27,1,1,36,42,37,288,40,140,13,5,0,31,0,1595,39792889,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,999999999,999999999,4,0,부동산업
1,20210801,100835,1,19850611,83,"26,170",NaN,NaN,1164149495,1211537593,178680349,985469146,1169589394,513029733,0,0,0,155910121,103544408,78096218,0,10366475,24032329,2375687088,2200605429,827091918,241416690,1409106858,84639004,804813397,1631905315,491000000,699170838,1190170838,251317068,0,-43552379,950487906,1397935527,1244231852,328029271,391473814,109359674,218669597,190070756,28598841,-9616467,-12035958,-87388699,0,47296386,28598841,-9616467,146863216,274886714,-99424657,-87388699,-106243560,-198191179,-19098050,-234699489,55606360,16,1,44693779,0,1,185,-96912068,159474035,98,2333738889,337057577,1170972000,7,117,59,141,33,-16,9,-27,33,58,-0,4,0,19,0,-4,0,-14,10,22,84,1,0,-7,67,23,-397,-18,-856,-6,16,0,3,0,99,3266777,0,0,0,0,0,0,0,0,0,0,0,0,

In [57]:
# 결측치 수 계산
null_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 필터링
null_counts = null_counts[null_counts > 0]

# 비율 계산 후 DataFrame으로 정리
null_df = pd.DataFrame({
    '컬럼명': null_counts.index,
    '결측치 수': null_counts.values,
    '결측치 비율 (%)': (null_counts.values / len(df) * 100).round(2)
})

# 보기 좋게 정렬
null_df = null_df.sort_values(by='결측치 수', ascending=False).reset_index(drop=True)

# 출력
null_df

,컬럼명,결측치 수,결측치 비율 (%)
0,상장폐지일자,91808,100
1,상장일자,91329,99
2,재무비율_총자산순이익률,26427,29
3,재무비율_당기순이익율,23616,26
4,주소지시군구,13541,15
5,재무비율_유형자산증가율,7351,8
6,재무비율_유동자산증가율,6402,7
7,재무비율_매입채무회전율,6401,7
8,EBITDA증가율,3978,4
9,영업이익이자보상배율,139,0


In [58]:
mask = df['재무비율_총자산순이익률'].isnull() & df['당기순이익'].notnull() & df['자산총계'].notnull()
df.loc[mask, '재무비율_총자산순이익률'] = (
    df.loc[mask, '당기순이익'] / df.loc[mask, '자산총계']
) * 100
mask = df['재무비율_당기순이익율'].isnull() & df['당기순이익'].notnull() & df['매출액'].notnull()
df.loc[mask, '재무비율_당기순이익율'] = (
    df.loc[mask, '당기순이익'] / df.loc[mask, '매출액']
) * 100
# 영업이익이자보상배율 = 영업손익 / 이자비용
mask = df['영업이익이자보상배율'].isnull() & df['영업손익'].notnull() & df['이자비용'].notnull()
df.loc[mask, '영업이익이자보상배율'] = df.loc[mask, '영업손익'] / df.loc[mask, '이자비용']
# 이자보상배율 = EBIT / 이자비용
mask = df['이자보상배율'].isnull() & df['EBIT'].notnull() & df['이자비용'].notnull()
df.loc[mask, '이자보상배율'] = df.loc[mask, 'EBIT'] / df.loc[mask, '이자비용']
# 매입채무회전율 = 매출원가 / 매입채무
mask = df['재무비율_매입채무회전율'].isnull() & df['매출원가'].notnull() & df['매입채무'].notnull()
df.loc[mask, '재무비율_매입채무회전율'] = df.loc[mask, '매출원가'] / df.loc[mask, '매입채무']
# 결측치 비율 큰 컬럼 삭제
df.drop(columns=['상장폐지일자', '상장일자'], inplace=True)

In [59]:
sido_map = {
    11: '서울', 26: '부산', 27: '대구', 28: '인천', 29: '광주', 30: '대전', 31: '울산',
    36: '세종', 41: '경기', 42: '강원', 43: '충북', 44: '충남', 45: '전북',
    46: '전남', 47: '경북', 48: '경남', 50: '제주'
}

# ① 주소지시군구가 null이 아닌 것만 사용
df = df[df['주소지시군구'].notnull()]

# ② 앞 2자리만 추출하고 숫자형으로 변환 (오류 발생 시 NaN 처리)
df['시도코드'] = df['주소지시군구'].astype(str).str[:2]
df['시도코드'] = pd.to_numeric(df['시도코드'], errors='coerce')  # <- 핵심!

# ③ 매핑 (숫자가 아니거나 범위 밖이면 NaN 됨)
df['시도명'] = df['시도코드'].map(sido_map)

# ④ 시도명 결측치 제거 (매핑 실패한 값들 제거)
df = df[df['시도명'].notnull()]

df.reset_index(drop=True, inplace=True)

C:\Users\campus4D006\AppData\Local\Temp\ipykernel_20936\328422276.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시도코드'] = df['주소지시군구'].astype(str).str[:2]
C:\Users\campus4D006\AppData\Local\Temp\ipykernel_20936\328422276.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시도코드'] = pd.to_numeric(df['시도코드'], errors='coerce')  # <- 핵심!
C:\Users\campus4D006\AppData\Local\Temp\ipykernel_20936\328422276.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [60]:
print("Before:", df.shape)
df.dropna(inplace=True)
print("After:", df.shape)

Before: (76731, 160)
After: (70751, 160)


In [61]:
# 결측치 수 계산
null_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 필터링
null_counts = null_counts[null_counts > 0]

# 비율 계산 후 DataFrame으로 정리
null_df = pd.DataFrame({
    '컬럼명': null_counts.index,
    '결측치 수': null_counts.values,
    '결측치 비율 (%)': (null_counts.values / len(df) * 100).round(2)
})

# 보기 좋게 정렬
null_df = null_df.sort_values(by='결측치 수', ascending=False).reset_index(drop=True)

# 출력
null_df

,컬럼명,결측치 수,결측치 비율 (%)


In [62]:
df_code = pd.read_csv('법정동코드 전체자료.txt', encoding='cp949', sep='\t')
df_code['법정동코드'] = df_code['법정동코드'].astype('str')
df_code

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49854,5280042024,전북특별자치도 부안군 위도면 대리,존재
49855,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49856,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49857,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [63]:
# 1. 주소지시군구 전처리: 문자열 + 소수점 제거 + zero-padding
df['주소지시군구'] = df['주소지시군구'].astype(str).str.replace('.0', '', regex=False).str.zfill(5)

# 2. 시도코드, 시군구코드 분리
df['시도코드'] = df['주소지시군구'].str[:2]
df['시군구코드'] = df['주소지시군구'].str[2:]

# 3. 법정동코드 기준 시도/시군구 코드 분리
df_code['시도코드'] = df_code['법정동코드'].str[:2]
df_code['시군구코드'] = df_code['법정동코드'].str[2:5]

# 4. 법정동명에서 시군구명 추출 (두 번째 단어)
df_code['시군구명'] = df_code['법정동명'].str.split().str[1]

# 5. 시도+시군구코드 기준으로 중복 제거 (가장 대표적인 시군구명만)
df_sgg_map = df_code[['시도코드', '시군구코드', '시군구명']].drop_duplicates(subset=['시도코드', '시군구코드'])

# 6. 병합 (Left Join, 행 증가 방지)
df = df.merge(df_sgg_map, on=['시도코드', '시군구코드'], how='left')

# 7. 세종시 예외 처리 (시군구가 없어서 NaN인 경우)
df.loc[(df['시도코드'] == '36') & (df['시군구명'].isnull()), '시군구명'] = '없음'

In [64]:
# 1. 시도-시군구 조합 유니크하게 추출
check_df = df[['시도명', '시군구명']].drop_duplicates()

# 2. 시도별 시군구명 개수 확인 (간단 시각 확인)
check_counts = check_df.groupby('시도명')['시군구명'].count().sort_values(ascending=False)
print(check_counts)

시도명
경기    31
서울    25
전남    20
경북    18
경남    16
부산    15
충남    14
전북    13
충북    10
인천     9
대구     8
광주     5
대전     5
울산     5
제주     2
세종     1
Name: 시군구명, dtype: int64


In [67]:
df.to_csv('결측치제거.csv', index=False, encoding='utf-8')

# 음수제거

In [ ]:
#비유동자산 -> 음수 제거
#매출채권 -> 음수 제거
#매출채권(전기) -> 음수 제거
#자산총계 -> 음수 제거
#유동부채 -> 음수 제거
#차입금 -> 음수 제거
#비유동부채 -> 음수 제거
#부채총계 -> 음수 제거
#자기자본(납입자본금) -> 음수 제거
#자본잉여금 -> 음수 제거
#납입자본 -> 음수 제거
#매출액 -> 음수 제거
#전기매출액 -> 음수 제거
#법인세 -> 음수 제거
#영업외수익 -> 음수 제거
#부채상환계수 -> 음수,0 제거
#영업이익이자보상배율 -> 음수,0 제거
#이자보상배율 -> 음수,0 제거
#재무비율_유동비율 -> 음수 제거
#재무비율_차입금의존도 -> 음수 제거
#재무비율_매출원가율 -> 음수 제거
#재무비율_판관비율 -> 음수 제거
#재무비율_매출채권회전율 -> 음수 제거
#재무비율_재고자산회전율 -> 음수 제거
#재무비율_매입채무회전율 -> 음수 제거
#재무비율_총자산회전율 -> 음수 제거
#단기차입금의존도 -> 음수 제거
#당좌비율 -> 음수 제거
#총자본회전율 -> 음수 제거
#소유건축물실거래가합계 -> 음수 제거
#차입금/EBITDA -> 음수 제거
#EBITDA/금융비용 -> 음수 제거

In [41]:
df = pd.read_csv('./결측치제거.csv',encoding='utf-8')
df

,기준년월,가명식별자,외감구분,설립일자,종업원수,주소지시군구,유동자산,비유동자산,당좌자산,재고자산,유형자산,재공품,현금,현금등가물,상품유가증권,현금성자산,매출채권,매출채권(전기),매출채권처분손실(당기),무형자산,투자자산,자산총계,자산총계(전기),유동부채,단기차입금,차입금,매입채무,비유동부채,부채총계,자기자본(납입자본금),자본잉여금,납입자본,이익잉여금,자본조정,기타포괄손익누계액,유보금,자본총계,전기자본총계,매출액,전기매출액,매출원가,매출총이익,판매비와관리비,법인세비용차감전 순이익,전기법인세차감전순이익,법인세,계속사업이익,중단산업손익,금융비용,영업손익,전기영업이익,영업외수익,영업외비용,법인세차감전순이익,당기순이익,당기순이익(전기),현금흐름,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,부채상환계수,영업이익이자보상배율,이자비용,사채이자(당기),이자보상배율,적립금비율,EBIT,EBITDA,청산가치율,청산가치,순운전자본,순차입금,재무비율_총자산증가율,재무비율_부채비율,재무비율_자기자본비율,재무비율_유동비율,재무비율_차입금의존도,재무비율_매출액증가율,재무비율_영업이익율,재무비율_당기순이익율,재무비율_매출원가율,재무비율_판관비율,재무비율_자기자본이익률(ROE),재무비율_매출채권회전율,재무비율_재고자산회전율,재무비율_매입채무회전율,재무비율_총자산회전율,재무비율_총자산순이익률,재무비율_유동자산증가율,재무비율_유형자산증가율,단기차입금의존도,당좌비율,순차입금비율,순운전자본회전율,총자본회전율,자기자본순이익율,매출총이익율,EBITDA마진율,영업이익증가율,당기순이익증가율,EBITDA증가율,OCF/매출액비용,부채상환계수.1,차입금/EBITDA,EBITDA/금융비용,사업장소유여부,소유건축물건수,소유건축물실거래가합계,사업장권리침해여부,소유건축물권리침해여부,기업신용공여연체과목수(일보)(미해제),기업신용공여연체과목수(일보)(3개월내유지)(해제포함),기업신용공여연체과목수(일보)(6개월내유지)(해제포함),기업신용공여연체과목수(일보)(1년내유지)(해제포함),기업신용공여연체과목수(일보)(3년내유지)(해제포함),기업신용공여연체과목수(일보)(3개월내발생)(해제포함),기업신용공여연체과목수(일보)(6개월내발생)(해제포함),기업신용공여연체과목수(일보)(1년내발생)(해제포함),기업신용공여연체과목수(일보)(3년내발생)(해제포함),기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함),기업신용공여30일이상연체과목수(일보)(해제포함),기업신용공여30일이상연체과목수(일보)(미해제),기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함),기업신용공여30일이상연체과목수(일보)(이자연체)(미해제),기업신용공여연체기관수(일보)(미해제),기업신용공여30일이상연체기관수(일보)(해제포함),기업신용공여30일이상연체기관수(일보)(미해제),기업신용공여30일이상연체기관수(일보)(연체)(해제포함),기업신용공여30일이상연체기관수(일보)(이자연체)(미해제),기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함),기업신용공여연체최장연체일수(일보)(3개월내유지)(해제포함),기업신용공여연체최장연체일수(일보)(6개월내유지)(해제포함),기업신용공여연체최장연체일수(일보)(1년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(3년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(5년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(3개월내발생)(해제포함),기업신용공여연체최장연체일수(일보)(6개월내발생)(해제포함),기업신용공여연체최장연체일수(일보)(1년내발생)(해제포함),기업신용공여연체최장연체일수(일보)(3년내발생)(해제포함),기업신용공여연체최장연체일수(일보)(5년내발생)(해제포함),신용도판단공공정보건수(CIS)(5년내발생)(해제포함),신용도판단정보공공정보건수(CIS)(미해제),신용도판단정보공공정보건수(관련인제외)(CIS)(당월유지)(해제포함),"공공정보(국세,지방세,관세체납)건수(CIS)(미해제)","공공정보(국세,지방세,관세체납)건수(CIS)(5년내발생)","공공정보(국세,지방세,관세체납,고용산재체납)건수(CIS)(미해제)","공공정보(국세,지방세,관세체납,고용산재체납)건수(CIS)(5년내발생)","신용도판단정보공공정보최근발생일자로부터경과일수(CIS)(해제,삭제)","신용도판단정보공공정보최근해제일자로부터경과일수(CIS)(해제,삭제)",기업신용평가등급(구간화),모형개발용Performance(향후1년내부도여부),업종(대분류),시도코드,시도명,시군구코드,시군구명
0,20210801,41294,1,20020830,63,11680,4406362501,2643245525,1684771733,2721590768,2126501078,67852456,0,0,0,24565275,515949640,909118333,0,0,478016874,7049608026,6735311108,1778620149,676428759,998606328,0,61218398,1839838547,256895000,0,256895000,2421662507,-12293178,-9379458,2421662507,2656884871,2425963213,2717487867,1790577446,1578601892,1138885975,154266460,984619515,253882089,59513932,926612530,0,31406384,984619515,253882089,96963470,95456523,986126462,926612530,664204007,-477677468,360587698,-1178644105,340378939,5,1,32788480,0,1,934,861217368,970761099,93,6532863579,2627742352,730196664,4,69,38,248,32,52,36,34,58,6,0,4,1,0,0,13,0,59,10,95,27,1,1,36,42,37,288,40,140,13,5,0,31,0,1595,39792889,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,999999999,999999999,4,0,부동산업,11,서울,680,강남구
1,20210801,100835,1,19850611,83,26170,1164149495,1211537593,178680349,985469146,1169589394,513029733,0,0,0,155910121,103544408,78096218,0,10366475,24032329,2375687088,2200605429,827091918,241416690,1409106858,84639004,804813397,1631905315,491000000,699170838,1190170838,251317068,0,-43552379,950487906,1397935527,1244231852,328029271,391473814,109359674,218669597,190070756,28598841,-9616467,-12035958,-87388699,0,47296386,28598841,-9616467,146863216,274886714,-99424657,-87388699,-106243560,-198191179,-19098050,-234699489,55606360,16,1,44693779,0,1,185,-96912068,159474035,98,2333738889,337057577,1170972000,7,117,59,141,33,-16,9,-27,33,58,-0,4,0,19,0,-4,0,-14,10,22,84,1,0,-7,67,23,-397,-18,-856,-6,16,0,3,0,99,3266777,0,0,0,0,0,0,0,0,0,0,0,0,

In [42]:
#  음수 제거만 수행할 컬럼 목록
cols_remove_negative = [
    '비유동자산', '자산총계', '유동부채', '차입금', '부채총계', 
    '자기자본(납입자본금)', '매출액', '전기매출액', '차입금/EBITDA', 'EBITDA/금융비용' 
]

#  음수 & 0 모두 제거할 컬럼 목록
cols_remove_nonpositive = ['이자보상배율']

#  음수 제거: 값이 0 이상인 행만 남김
for col in cols_remove_negative:
    if col in df.columns:
        df = df[df[col] >= 0]

#  음수 & 0 제거: 값이 0 초과인 행만 남김
for col in cols_remove_nonpositive:
    if col in df.columns:
        df = df[df[col] > 0]

In [47]:
# 음수 제거만 한 컬럼들 → 최소값이 0 이상이어야 정상
print("✅ 음수 제거 대상 컬럼 최소값:")
print(df[cols_remove_negative].min())

# 음수 + 0 제거한 컬럼들 → 최소값이 0 초과여야 정상
print("\n✅ 음수 + 0 제거 대상 컬럼 최소값:")
print(df[cols_remove_nonpositive].min())

✅ 음수 제거 대상 컬럼 최소값:
비유동자산              0
자산총계          17,853
유동부채              10
차입금                0
부채총계           3,596
자기자본(납입자본금)        0
매출액                0
전기매출액              0
차입금/EBITDA         0
EBITDA/금융비용       -0
dtype: float64

✅ 음수 + 0 제거 대상 컬럼 최소값:
이자보상배율   0
dtype: float64


In [46]:
df.to_csv('final_결측치제거2.csv',index=False,encoding='utf-8')
df

,기준년월,가명식별자,외감구분,설립일자,종업원수,주소지시군구,유동자산,비유동자산,당좌자산,재고자산,유형자산,재공품,현금,현금등가물,상품유가증권,현금성자산,매출채권,매출채권(전기),매출채권처분손실(당기),무형자산,투자자산,자산총계,자산총계(전기),유동부채,단기차입금,차입금,매입채무,비유동부채,부채총계,자기자본(납입자본금),자본잉여금,납입자본,이익잉여금,자본조정,기타포괄손익누계액,유보금,자본총계,전기자본총계,매출액,전기매출액,매출원가,매출총이익,판매비와관리비,법인세비용차감전 순이익,전기법인세차감전순이익,법인세,계속사업이익,중단산업손익,금융비용,영업손익,전기영업이익,영업외수익,영업외비용,법인세차감전순이익,당기순이익,당기순이익(전기),현금흐름,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,부채상환계수,영업이익이자보상배율,이자비용,사채이자(당기),이자보상배율,적립금비율,EBIT,EBITDA,청산가치율,청산가치,순운전자본,순차입금,재무비율_총자산증가율,재무비율_부채비율,재무비율_자기자본비율,재무비율_유동비율,재무비율_차입금의존도,재무비율_매출액증가율,재무비율_영업이익율,재무비율_당기순이익율,재무비율_매출원가율,재무비율_판관비율,재무비율_자기자본이익률(ROE),재무비율_매출채권회전율,재무비율_재고자산회전율,재무비율_매입채무회전율,재무비율_총자산회전율,재무비율_총자산순이익률,재무비율_유동자산증가율,재무비율_유형자산증가율,단기차입금의존도,당좌비율,순차입금비율,순운전자본회전율,총자본회전율,자기자본순이익율,매출총이익율,EBITDA마진율,영업이익증가율,당기순이익증가율,EBITDA증가율,OCF/매출액비용,부채상환계수.1,차입금/EBITDA,EBITDA/금융비용,사업장소유여부,소유건축물건수,소유건축물실거래가합계,사업장권리침해여부,소유건축물권리침해여부,기업신용공여연체과목수(일보)(미해제),기업신용공여연체과목수(일보)(3개월내유지)(해제포함),기업신용공여연체과목수(일보)(6개월내유지)(해제포함),기업신용공여연체과목수(일보)(1년내유지)(해제포함),기업신용공여연체과목수(일보)(3년내유지)(해제포함),기업신용공여연체과목수(일보)(3개월내발생)(해제포함),기업신용공여연체과목수(일보)(6개월내발생)(해제포함),기업신용공여연체과목수(일보)(1년내발생)(해제포함),기업신용공여연체과목수(일보)(3년내발생)(해제포함),기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함),기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함),기업신용공여30일이상연체과목수(일보)(해제포함),기업신용공여30일이상연체과목수(일보)(미해제),기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함),기업신용공여30일이상연체과목수(일보)(이자연체)(미해제),기업신용공여연체기관수(일보)(미해제),기업신용공여30일이상연체기관수(일보)(해제포함),기업신용공여30일이상연체기관수(일보)(미해제),기업신용공여30일이상연체기관수(일보)(연체)(해제포함),기업신용공여30일이상연체기관수(일보)(이자연체)(미해제),기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함),기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함),기업신용공여연체최장연체일수(일보)(3개월내유지)(해제포함),기업신용공여연체최장연체일수(일보)(6개월내유지)(해제포함),기업신용공여연체최장연체일수(일보)(1년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(3년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(5년내유지)(해제포함),기업신용공여연체최장연체일수(일보)(3개월내발생)(해제포함),기업신용공여연체최장연체일수(일보)(6개월내발생)(해제포함),기업신용공여연체최장연체일수(일보)(1년내발생)(해제포함),기업신용공여연체최장연체일수(일보)(3년내발생)(해제포함),기업신용공여연체최장연체일수(일보)(5년내발생)(해제포함),신용도판단공공정보건수(CIS)(5년내발생)(해제포함),신용도판단정보공공정보건수(CIS)(미해제),신용도판단정보공공정보건수(관련인제외)(CIS)(당월유지)(해제포함),"공공정보(국세,지방세,관세체납)건수(CIS)(미해제)","공공정보(국세,지방세,관세체납)건수(CIS)(5년내발생)","공공정보(국세,지방세,관세체납,고용산재체납)건수(CIS)(미해제)","공공정보(국세,지방세,관세체납,고용산재체납)건수(CIS)(5년내발생)","신용도판단정보공공정보최근발생일자로부터경과일수(CIS)(해제,삭제)","신용도판단정보공공정보최근해제일자로부터경과일수(CIS)(해제,삭제)",기업신용평가등급(구간화),모형개발용Performance(향후1년내부도여부),업종(대분류),시도코드,시도명,시군구코드,시군구명
0,20210801,41294,1,20020830,63,11680,4406362501,2643245525,1684771733,2721590768,2126501078,67852456,0,0,0,24565275,515949640,909118333,0,0,478016874,7049608026,6735311108,1778620149,676428759,998606328,0,61218398,1839838547,256895000,0,256895000,2421662507,-12293178,-9379458,2421662507,2656884871,2425963213,2717487867,1790577446,1578601892,1138885975,154266460,984619515,253882089,59513932,926612530,0,31406384,984619515,253882089,96963470,95456523,986126462,926612530,664204007,-477677468,360587698,-1178644105,340378939,5,1,32788480,0,1,934,861217368,970761099,93,6532863579,2627742352,730196664,4,69,38,248,32,52,36,34,58,6,0,4,1,0,0,13,0,59,10,95,27,1,1,36,42,37,288,40,140,13,5,0,31,0,1595,39792889,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,999999999,999999999,4,0,부동산업,11,서울,680,강남구
1,20210801,100835,1,19850611,83,26170,1164149495,1211537593,178680349,985469146,1169589394,513029733,0,0,0,155910121,103544408,78096218,0,10366475,24032329,2375687088,2200605429,827091918,241416690,1409106858,84639004,804813397,1631905315,491000000,699170838,1190170838,251317068,0,-43552379,950487906,1397935527,1244231852,328029271,391473814,109359674,218669597,190070756,28598841,-9616467,-12035958,-87388699,0,47296386,28598841,-9616467,146863216,274886714,-99424657,-87388699,-106243560,-198191179,-19098050,-234699489,55606360,16,1,44693779,0,1,185,-96912068,159474035,98,2333738889,337057577,1170972000,7,117,59,141,33,-16,9,-27,33,58,-0,4,0,19,0,-4,0,-14,10,22,84,1,0,-7,67,23,-397,-18,-856,-6,16,0,3,0,99,3266777,0,0,0,0,0,0,0,0,0,0,0,0,